In [ ]:
import os
import openai
openai.api_key = ""
# openai.Model.list()

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.7, # default is 1 this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
import json
import pandas as pd
from nltk.tokenize import sent_tokenize
import ast
import time
import json
import pandas as pd
from nltk.tokenize import sent_tokenize

data = json.load(open("factchecks.json", 'r'))

In [ ]:
for i in range(len(data)):
    print(i)
    text = data[i]['claim']
    prompt = f"""
    Generate ten different tweets about the text delimited by triple backticks.
    Make sure that generated tweets should not be a fact-check or a debunk.
    Also, tweets should have different hashtags.
    ```{text}```
    """
    max_attempts = 5
    for j in range(max_attempts):
        try:
            response = get_completion(prompt)
            break
        except Exception as e:
            time.sleep(2)
            print(f"Error occurred: {e}")
            print(f"Retrying ({j+1}/{max_attempts})...")
    if j == max_attempts - 1:
        print("Maximum retries exceeded. Stopping at ", i)
        break
        
    data[i]['chatgpt_gen'] = response

In [ ]:
import re
for i in range(len(data)):
    text = "\n"+data[i]['chatgpt_gen']
    if "1)" in data[i]['chatgpt_gen'] and "2)" in data[i]['chatgpt_gen'] and "3)" in data[i]['chatgpt_gen']:
        items = re.split(r'\n\d+\)\s+', text)[1:]
    elif "Tweet 1:" in data[i]['chatgpt_gen'] and "Tweet 2:" in data[i]['chatgpt_gen'] and "Tweet 3:" in data[i]['chatgpt_gen']:
        items = re.split(r'\nTweet \d+\:\s+', text)[1:]
    else:
        items = re.split(r'\n\d+\.\s+', text)[1:]
    items = [item.strip() for item in items]
    items = [item.strip('"') for item in items]
    if len(items) != 5: print(i)
    data[i]['chatgpt_gen_list'] = items

In [ ]:
with open('CahtGPTMisinfo.json', 'w') as fp:
    json.dump(data, fp)

In [ ]:
# computing selfBLEU 

In [ ]:
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
import copy
metric = load_metric("bleu")

In [ ]:
final_score1 = []
final_score2 = []
final_score3 = []
from tqdm import tqdm

for i in tqdm(range(len(data))):
    p = data[i]['chatgpt_gen'][:5]
    def calculate_selfBleu(sentences, order):
        def get_bleu_score(sentence, remaining_sentences, order):
            lst = []
            for i in remaining_sentences:
                preds = [sentence.lower().split()]
                labels = [[i.lower().split()]]
                bleu = metric.compute(predictions=preds, references=labels, max_order=int(order))
                lst.append(bleu['bleu'])
            return lst
        bleu_scores = []
        for i in sentences:
            sentences_copy = copy.deepcopy(sentences)
            remaining_sentences = sentences_copy.remove(i)
            bleu = get_bleu_score(i, sentences_copy, order)
            bleu_scores.append(bleu)
        return np.mean(bleu_scores)
    
    final_score1.append(calculate_selfBleu(p, 1))
    final_score2.append(calculate_selfBleu(p, 2))
    final_score3.append(calculate_selfBleu(p, 3))
    
print(sum(final_score1)/len(final_score1))
print(sum(final_score2)/len(final_score2))
print(sum(final_score3)/len(final_score3))

In [ ]:
final_score1 = []
final_score2 = []
final_score3 = []
from tqdm import tqdm

for i in tqdm(range(len(data))):
    p = data[i]['generatedT5Misinfo'][:5]
    def calculate_selfBleu(sentences, order):
        def get_bleu_score(sentence, remaining_sentences, order):
            lst = []
            for i in remaining_sentences:
                preds = [sentence.lower().split()]
                labels = [[i.lower().split()]]
                bleu = metric.compute(predictions=preds, references=labels, max_order=int(order))
                lst.append(bleu['bleu'])
            return lst
        bleu_scores = []
        for i in sentences:
            sentences_copy = copy.deepcopy(sentences)
            remaining_sentences = sentences_copy.remove(i)
            bleu = get_bleu_score(i, sentences_copy, order)
            bleu_scores.append(bleu)
        return np.mean(bleu_scores)
    
    final_score1.append(calculate_selfBleu(p, 1))
    final_score2.append(calculate_selfBleu(p, 2))
    final_score3.append(calculate_selfBleu(p, 3))